In [1]:
import numpy as np
import pandas as pd

**EDA**

In [2]:
df = pd.read_csv('../input/fake-news-prediction/train.csv')

In [3]:
df.head()

In [4]:
df.shape

In [5]:
df.describe()

In [6]:
df.info()

In [7]:
df.isnull().sum()

In [8]:
# Drop Nan Values
df = df.dropna()

In [9]:
df.head(20)

In [10]:
# Get The Independent  Features
X = df.drop('label',axis=1)

In [11]:
print(X)

In [12]:
X.shape

In [13]:
# Get The Dependent Feature
y = df['label']

In [14]:
print(y)

In [15]:
y.shape

In [16]:
y.value_counts()

In [17]:
import tensorflow as tf

In [18]:
tf.__version__

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

**One Hot Representation**

In [20]:
# Vocablary Size
voc_size = 5000

In [21]:
dataset = X.copy()

In [22]:
dataset['title'][1]

In [23]:
dataset

In [24]:
dataset.reset_index(inplace = True)

In [25]:
dataset

In [26]:
import nltk
import re
from nltk.corpus import stopwords

In [27]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
corpus

In [29]:
onehot_rep =[one_hot(words,voc_size) for words in corpus]

In [30]:
onehot_rep

**Embedding Representation**

In [31]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding = 'post',maxlen = sent_length)
print(embedded_docs)

In [32]:
embedded_docs[0]

**Creating Bidirectional LSTM Model**

In [33]:
embedding_features = 40 #For Features In Every Word In Embedded_docs
model = Sequential()
model.add(Embedding(voc_size,embedding_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'adam',loss= 'binary_crossentropy',metrics=['accuracy'])

In [34]:
model.summary()

In [35]:
len(embedded_docs),y.shape

In [36]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [37]:
X_final.shape , y_final.shape

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size = 0.33,random_state = 42)

**Model Training**

In [39]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 20 ,batch_size= 64)

**Performance Metrics And Accuracy**

In [40]:
y_pred = model.predict(X_test)

In [41]:
y_pred = np.where(y_pred>0.5,1,0) #AUC ROC Curve

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))